In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
from skrl.envs.wrappers.torch import wrap_env
from skrl.models.torch import DeterministicMixin, Model, MultiCategoricalMixin
from skrl.trainers.torch import ParallelTrainer
from skrl.utils import set_seed
from torch.nn import TransformerEncoder, TransformerEncoderLayer

from preprocess import preprocess
from skrl.utils.spaces.torch import unflatten_tensorized_space

# set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="dict_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

In [ ]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/futures/15m/2024/**/*.pkl",
    preprocess=preprocess,
    positions=[-1, 0, 1],
    multiplier=[5, 20, 50],
    trading_fees=0.0001,
    borrow_interest_rate=0.0003,
    portfolio_initial_value=1000,
    max_episode_duration="max",
    verbose=1,
    window_size=60,
    btc_index=True,
)

In [ ]:
env = gym.make(**env_cfg)
obs = env.observation_space
env = wrap_env(env, wrapper="gymnasium")

In [ ]:
device = env.device
replay_buffer_size = 1024 * 1 * env.num_envs
memory_size = int(replay_buffer_size / env.num_envs)

In [ ]:
class LearnablePositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1024 * 16):
        super().__init__()
        self.position_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        positions = positions.expand(batch_size, seq_len)
        position_encoded = self.position_embedding(positions)
        return x + position_encoded


class AttentionPooling(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionPooling, self).__init__()
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, x):
        attention_weights = self.attention(x).squeeze(-1)
        attention_weights = torch.softmax(attention_weights, dim=-1)
        pooled_x = torch.bmm(attention_weights.unsqueeze(1), x).squeeze(1)
        return pooled_x


class SharedNoFC(MultiCategoricalMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        unnormalized_log_prob=True,
        reduction="sum",
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultiCategoricalMixin.__init__(self, unnormalized_log_prob, reduction)
        DeterministicMixin.__init__(self, clip_actions)

        self._shared_features = None
        self.num_features = 8
        self.num_infos = 8

        # Transformer Encoder for self-attention
        transformer_layer = TransformerEncoderLayer(
            d_model=self.num_features,  # The size of the input feature vector
            nhead=4,  # Number of attention heads
            dim_feedforward=256,  # The size of the feedforward network in the encoder
            dropout=0.1,
            batch_first=True,  # Use batch_first for better inference performance
        )
        self.positional_encoding = LearnablePositionalEncoding(
            d_model=self.num_features
        )
        self.transformer_encoder = TransformerEncoder(transformer_layer, num_layers=2)
        self.attention_pooling = AttentionPooling(self.num_features)

        self.shared_head = nn.Sequential(
            nn.LayerNorm(8),
            nn.GELU(),
            nn.Dropout(0.2),
        )

        self.info_head = nn.Sequential(
            nn.Linear(self.num_infos, 8),
            nn.LayerNorm(8),
            nn.ELU(),
            nn.Dropout(0.2),
        )

        self.policy_head = nn.Sequential(nn.Linear(self.num_features + self.num_infos, self.num_actions))
        self.value_head = nn.Sequential(nn.Linear(self.num_features + self.num_infos, 1))

    def act(self, inputs, role):
        return MultiCategoricalMixin.act(self, inputs, role)

    def compute(self, inputs, role):
        states = unflatten_tensorized_space(obs, inputs["states"])
        features = states["features"]  # (batch_size, seq_length, num_features=8)
        infos = states["infos"] # (batch_size, num_infos=6)

        features = self.positional_encoding(features)
        features = self.transformer_encoder(features)
        features = self.attention_pooling(features)

        _shared_features = self.shared_head(features)
        _shared_infos = self.info_head(infos)

        self._fusion = torch.cat([_shared_features, _shared_infos], dim=-1)

        actions = self.policy_head(self._fusion)

        return actions, {}


In [ ]:
models = {}
models["policy"] = SharedNoFC(env.observation_space, env.action_space, device)

for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)

In [ ]:
from skrl.agents.torch.ppo import PPO_DEFAULT_CONFIG
from skrl.resources.schedulers.torch import KLAdaptiveLR
import warnings

from skrl.agents.torch.ppo import PPO

cfg = PPO_DEFAULT_CONFIG.copy()
cfg["rollouts"] = memory_size
cfg["learning_epochs"] = 16
cfg["mini_batches"] = 8
cfg["discount_factor"] = 0.99
cfg["learning_rate"] = 5e-4
cfg["learning_rate_scheduler"] = KLAdaptiveLR
cfg["learning_rate_scheduler_kwargs"] = {
    "kl_threshold": 0.01,
    "min_lr": 1e-7,
    "max_lr": 1e-3,
}
cfg["mixed_precision"] = True

cfg["experiment"]["write_interval"] = 20000
cfg["experiment"]["checkpoint_interval"] = 100000
cfg["experiment"]["directory"] = "runs/torch/mddt"

In [ ]:
class PPO_EVAL(PPO):
    def act(self, states: torch.Tensor, timestep: int, timesteps: int) -> torch.Tensor:
        if timestep < self._random_timesteps:
            return self.policy.random_act({"states": self._state_preprocessor(states)}, role="policy")

        with torch.autocast(device_type=self._device_type, enabled=self._mixed_precision):
            actions, log_prob, outputs = self.policy.act({"states": self._state_preprocessor(states)}, role="policy")
            self._current_log_prob = log_prob

        dist = self.policy.distribution(role="policy")
        actions = dist.mean

        log_prob = dist.log_prob(actions)
        return actions, log_prob, outputs

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

agent = PPO(
    models=models,
    cfg=cfg,
    observation_space=env.observation_space,
    action_space=env.action_space,
    device=device,
)
path = "24-12-19_08-00-52-081885_PPO"
agent.load(f"/home/pitin/Desktop/hp/runs/torch/mddt/{path}/checkpoints/best_agent.pt")
cfg_trainer = {"timesteps": 10000, "headless": True}
trainer = ParallelTrainer(cfg=cfg_trainer, env=env, agents=[agent])

In [ ]:
trainer.eval()

In [ ]:
states, infos = env.reset()
timestep = 0
timesteps = 3000
terminated = torch.tensor([[False] * env.num_envs])
print(infos["portfolio_valuation"])
# while (not terminated.any()):
for t in range(timesteps):
    agent.pre_interaction(timestep=t, timesteps=timesteps)

    with torch.no_grad():
        actions = agent.act(states, timestep=t, timesteps=timesteps)[0]
        next_states, rewards, terminated, truncated, infos = env.step(actions)
        env.render()

    super(type(agent), agent).post_interaction(timestep=t, timesteps=timesteps)

    if env.num_envs > 1:
        states = next_states
    else:
        if terminated.any() or truncated.any():
            with torch.no_grad():
                states, infos = env.reset()
        else:
            states = next_states
print(infos["portfolio_valuation"])
# env.save_for_render()

In [ ]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()